<a href="https://colab.research.google.com/github/min02yam/R-study/blob/main/%EC%97%B0%EC%8A%B5/caret_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


2022.06.14
# caret
**3 Pre-Processing** <br> 
1. 더미 코딩
2. Zero- and Near Zero-Variance Predictors
3. correlation 예측변수 식별
4. Linear Dependencies
5. centering and scaling


In [9]:
#install.packages(c("caret", "pacman"))
install.packages(c("dplyr","tidyverse","skimr","DataExplorer"))
pacman::p_load("caret","dplyr","tidyverse","skimr","DataExplorer" )

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘htmlwidgets’, ‘igraph’, ‘gridExtra’, ‘networkD3’




# 2. Zero- and Near Zero-Variance Predictors

In [6]:
data(mdrr)

In [32]:
skim(mdrrDescr)

── Data Summary ────────────────────────
                           Values   
Name                       mdrrDescr
Number of rows             528      
Number of columns          342      
_______________________             
Column type frequency:              
  numeric                  342      
________________________            
Group variables            None     

── Variable type: numeric ──────────────────────────────────────────────────────
    skim_variable n_missing complete_rate          mean            sd      p0
  1 MW                    0             1    374.            93.4     121.   
  2 AMW                   0             1      7.02           0.768     4.85 
  3 Sv                    0             1     32.9            7.82     12.0  
  4 Se                    0             1     53.3           13.6      19.6  
  5 Sp                    0             1     34.8            8.31     12.8  
  6 Ss                    0             1     57.5           16.4      17.8 

In [15]:
mdrrDescr$nR11 %>% table %>% data.frame

.,Freq
<fct>,<int>
0,501
1,4
2,23


In [28]:
nzv =  nearZeroVar(mdrrDescr, saveMetrics= TRUE)
nzv[nzv$nzv,][1:10,]  # TRUE인 것만 찾아냄

,freqRatio,percentUnique,zeroVar,nzv
,<dbl>,<dbl>,<lgl>,<lgl>
nTB,23.00000,0.3787879,FALSE,TRUE
nBR,131.00000,0.3787879,FALSE,TRUE
nI,527.00000,0.3787879,FALSE,TRUE
nR03,527.00000,0.3787879,FALSE,TRUE
nR08,527.00000,0.3787879,FALSE,TRUE
nR11,21.78261,0.5681818,FALSE,TRUE
nR12,57.66667,0.3787879,FALSE,TRUE
D.Dr03,527.00000,0.3787879,FALSE,TRUE
D.Dr07,123.50000,5.8712121,FALSE,TRUE


In [25]:
dim(mdrrDescr)

[1] 528 342

In [30]:
nzv = nearZeroVar(mdrrDescr)
nzv

[1]  22  31  32  34  38  41  42 259 262 263 264 266 267 270 271 272 273 274 276
[20] 277 278 279 280 281 282 283 284 285 286 287 288 327 328 330 331 333 334 335
[39] 336 337 338 339 340 341 342

In [31]:
#열제거 
filteredDescr = mdrrDescr[, -nzv]
dim(filteredDescr)

[1] 528 297

# 3.  correlation 예측변수 식별
findCorrelation

In [37]:
cor= cor(filteredDescr) 

In [51]:
abs(cor[cor %>% upper.tri]) > .999 -> abs
abs %>% sum -> highcorr

In [58]:
cor[upper.tri(cor)] %>% summary

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-0.99607 -0.05373  0.25006  0.26078  0.65527  1.00000 

In [74]:
install.packages("magrittr")
library(magrittr)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract




In [75]:
highcorr75= findCorrelation(cor, cutoff=.75) # 열번호 생성
filteredDescr %<>% select(! highcorr75  )

In [76]:
#correlation을 다시 보면 확인 가능 
descrCor2 = cor(filteredDescr)
summary(descrCor2[upper.tri(descrCor2)])

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
-0.70728 -0.05378  0.04418  0.06692  0.18858  0.74458 

# 4. Linear Dependencies
findLinearCombos

In [78]:
ltfrDesign = matrix(0, nrow=6, ncol=6)
ltfrDesign[,1] <- c(1, 1, 1, 1, 1, 1)
ltfrDesign[,2] <- c(1, 1, 1, 0, 0, 0)
ltfrDesign[,3] <- c(0, 0, 0, 1, 1, 1)
ltfrDesign[,4] <- c(1, 0, 0, 1, 0, 0)
ltfrDesign[,5] <- c(0, 1, 0, 0, 1, 0)
ltfrDesign[,6] <- c(0, 0, 1, 0, 0, 1)
ltfrDesign

1,1,0,1,0,0
1,1,0,0,1,0
1,1,0,0,0,1
1,0,1,1,0,0
1,0,1,0,1,0
1,0,1,0,0,1


In [79]:
comboInfo = findLinearCombos(ltfrDesign)
comboInfo

$linearCombos
$linearCombos[[1]]
[1] 3 1 2

$linearCombos[[2]]
[1] 6 1 4 5


$remove
[1] 3 6

In [82]:
ltfrDesign[, -comboInfo$remove]

1,1,1,0
1,1,0,1
1,1,0,0
1,0,1,0
1,0,0,1
1,0,0,0


# 5. centering and scaling
preProcess

In [91]:
set.seed(2022)
#sample
ind=sample(seq(along=mdrrClass), length(mdrrClass)/2 )
train= filteredDescr[ind,]
test= filteredDescr[-ind,]
train_MDRR= mdrrClass[ind]
test_MDRR= mdrrClass[-ind]

In [96]:
standard= preProcess(train, method = c("center","scale"))
train_trans= predict( standard, train)
test_trans= predict(standard, test)

In [97]:
train_trans %>% summary

      AMW                Mp                 Ms               nDB         
 Min.   :-2.0992   Min.   :-2.20803   Min.   :-1.7715   Min.   :-0.8366  
 1st Qu.:-0.7745   1st Qu.:-0.78105   1st Qu.:-0.7490   1st Qu.:-0.8366  
 Median :-0.1070   Median :-0.06757   Median :-0.1099   Median :-0.8366  
 Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.5328   3rd Qu.: 0.64592   3rd Qu.: 0.4546   3rd Qu.: 0.3445  
 Max.   : 4.4723   Max.   : 4.57011   Max.   : 3.7671   Max.   : 2.7066  
      nAB                nS                nF               nCL         
 Min.   :-2.6009   Min.   :-0.3833   Min.   :-0.2709   Min.   :-0.4191  
 1st Qu.:-0.2314   1st Qu.:-0.3833   1st Qu.:-0.2709   1st Qu.:-0.4191  
 Median :-0.2314   Median :-0.3833   Median :-0.2709   Median :-0.4191  
 Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.5585   3rd Qu.:-0.3833   3rd Qu.:-0.2709   3rd Qu.:-0.4191  
 Max.   : 3.1913   Max.   : 4.4348   Max.   

In [ ]:
min_max_norm= preProcess(train, method = c("range"))
train_mmn=predict(min_max_norm, train)
train_mmn %>% summary

In [102]:
min_max= function(x){
  (x-min(x))/ (max(x)-min(x))
}

In [103]:
train %>% mutate_if(is.numeric, min_max) -> train_mmn2

In [ ]:
train_mmn2 %>% summary

In [105]:
test %>% mutate_if(is.numeric, min_max) -> test_mmn2
predict(min_max_norm, test) -> test_mmn

In [107]:
test_mmn %>% skim
test_mmn2 %>% skim

── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             264       
Number of columns          50        
_______________________              
Column type frequency:               
  numeric                  50        
________________________             
Group variables            None      

── Variable type: numeric ──────────────────────────────────────────────────────
   skim_variable n_missing complete_rate      mean        sd            p0
 1 AMW                   0             1 0.343     0.170     -0.126       
 2 Mp                    0             1 0.344     0.159     -0.0526      
 3 Ms                    0             1 0.327     0.172     -0.0692      
 4 nDB                   0             1 0.240     0.267      0           
 5 nAB                   0             1 0.460     0.181     -0.136       
 6 nS                    0             1 0.0758    0.204      0           
 7 nF     

http://topepo.github.io/caret/pre-processing.html